In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import torch
from sklearn.model_selection import train_test_split
from sklearn import metrics as skmetrics
import glob

In [2]:
train_data = ['../data-collection/dataset/acne-severity/NNEW_trainval_0.txt',
            '../data-collection/dataset/acne-severity/NNEW_trainval_1.txt',
            '../data-collection/dataset/acne-severity/NNEW_trainval_2.txt',
            '../data-collection/dataset/acne-severity/NNEW_trainval_3.txt',
            '../data-collection/dataset/acne-severity/NNEW_trainval_4.txt']
test_data = ['../data-collection/dataset/acne-severity/NNEW_test_0.txt',
            '../data-collection/dataset/acne-severity/NNEW_test_1.txt',
            '../data-collection/dataset/acne-severity/NNEW_test_2.txt',
            '../data-collection/dataset/acne-severity/NNEW_test_3.txt',
            '../data-collection/dataset/acne-severity/NNEW_test_4.txt']
path = '../data-collection/dataset/acne-severity/JPEGImages'

In [3]:
class Classification_Dataset:
    def __init__(self, data, data_path, transform, training=True):
        self.data = data
        self.imgs = data['path'].unique().tolist()
        self.data_path = data_path
        self.training = training
        self.transform = transform
    def __getitem__(self, idx):
        img = Image.open(os.path.join(self.data_path, self.data.iloc[idx, 0]))

        if(self.training):
            label = self.data.iloc[idx, 1]
        if self.transform is not None:
            img = self.transform(img)
        if(self.training):
            return img, label
        else:
            return img

    def __len__(self):
        return len(self.imgs)

In [4]:
def make_loader(dataset, train_batch_size, validation_split=0.2):
    # number of samples in train and test set
    train_len = int(len(dataset) * (1 - validation_split))
    test_len = len(dataset) - train_len
    train_set, test_set = torch.utils.data.random_split(dataset, [train_len, test_len])
    # create train_loader
    print(len(train_set))
    train_loader = torch.utils.data.DataLoader(
        train_set, batch_size=train_batch_size, shuffle=True,
    )
    # create test_loader
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False,)
    return train_loader, test_loader

In [5]:
def data_split(data, test_size):
    x_train, x_test, y_train, y_test = train_test_split(
        data, data["label"], test_size=test_size, stratify = data.iloc[:,1]
    )
    return x_train, x_test, y_train, y_test

In [6]:
class Metrics:
    def __init__(self, metric_names):
        self.metric_names = metric_names
        # initialize a metric dictionary
        self.metric_dict = {metric_name: [0] for metric_name in self.metric_names}

    def step(self, labels, preds):
        for metric in self.metric_names:
            # get the metric function
            do_metric = getattr(
                skmetrics, metric, "The metric {} is not implemented".format(metric)
            )
            # check if metric require average method, if yes set to 'micro' or 'macro' or 'None'
            try:
                self.metric_dict[metric].append(
                    do_metric(labels, preds, average="macro")
                )
            except:
                self.metric_dict[metric].append(do_metric(labels, preds))

    def epoch(self):
        # calculate metrics for an entire epoch
        avg = [sum(metric) / (len(metric) - 1) for metric in self.metric_dict.values()]
        metric_as_dict = dict(zip(self.metric_names, avg))
        return metric_as_dict

    def last_step_metrics(self):
        # return metrics of last steps
        values = [self.metric_dict[metric][-1] for metric in self.metric_names]
        metric_as_dict = dict(zip(self.metric_names, values))
        return metric_as_dict

In [7]:
import torch.nn.functional as F
import torch.nn as nn

class LabelSmoothingLoss(nn.Module):
    def __init__(self, smoothing=0.1, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.smoothing = smoothing
        self.dim = dim

    def forward(self, pred, target):
        target = F.one_hot(target, num_classes=pred.size(-1))
        target = target.float()
        target = (1 - self.smoothing) * target + self.smoothing / pred.size(-1)
        log_pred = F.log_softmax(pred, dim=self.dim)
        loss = nn.KLDivLoss(reduction='batchmean')(log_pred, target)
        return loss
    
criterion = LabelSmoothingLoss(smoothing=0.12)

In [8]:
def train_one_epoch(model, train_loader, test_loader, optimizer, criterion, train_metrics, val_metrics,):
    # training-the-model
    train_loss = 0
    valid_loss = 0
    all_labels = []
    all_preds = []
    model.train()
    for data, target in train_loader:
        # move-tensors-to-GPU
        data = data.type(torch.FloatTensor)
        # day = day.view(-1,1).type(torch.FloatTensor)
        # target=torch.Tensor(target)
        target = target.float()
        # clear-the-gradients-of-all-optimized-variables
        optimizer.zero_grad()
        # forward-pass: compute-predicted-outputs-by-passing-inputs-to-the-model
        output = model(data)
        preds = torch.argmax(output, axis=1).cpu().detach().numpy()
        labels = target.cpu().numpy()
        # calculate-the-batch-loss
        loss = criterion(output.type(torch.FloatTensor), target.type(torch.LongTensor))
        # backward-pass: compute-gradient-of-the-loss-wrt-model-parameters
        loss.backward()
        # perform-a-ingle-optimization-step (parameter-update)
        optimizer.step()
        # update-training-loss
        train_loss += loss.item() * data.size(0)
        # calculate training metrics
        all_labels.extend(labels)
        all_preds.extend(preds)
    
    train_metrics.step(all_labels, all_preds)

    # validate-the-model
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for data, target in test_loader:
            data = data.type(torch.FloatTensor)
            # day = day.view(-1,1).type(torch.FloatTensor)
            output = model(data)
            preds = torch.argmax(output, axis=1).tolist()
            labels = target.tolist()
            all_labels.extend(labels)
            all_preds.extend(preds)
            loss = criterion(output, target)

            # update-average-validation-loss
            valid_loss += loss.item() * data.size(0)

    val_metrics.step(all_labels, all_preds)
    train_loss = train_loss / len(train_loader.sampler)
    valid_loss = valid_loss / len(test_loader.sampler)

    return (
        train_loss,
        valid_loss,
        train_metrics.last_step_metrics(),
        val_metrics.last_step_metrics(),
    )

In [9]:
bs = 20

In [10]:
train_df = pd.read_csv(train_data[0], names=['path','label','leisons'], sep='  ')

C:\Users\prais\AppData\Local\Temp\ipykernel_21464\269451781.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  train_df = pd.read_csv(train_data[0], names=['path','label','leisons'], sep='  ')


In [11]:
train_df

,path,label,leisons
0,levle0_0.jpg,0,3
1,levle0_1.jpg,0,3
2,levle0_100.jpg,0,2
3,levle0_101.jpg,0,1
4,levle0_102.jpg,0,3
...,...,...,...
1160,levle3_92.jpg,3,63
1161,levle3_93.jpg,3,61
1162,levle3_94.jpg,3,58
1163,levle3_95.jpg,3,58


In [12]:
x_train, x_val, y_train, y_val = data_split(train_df,0.2)

In [13]:
test_df = pd.read_csv(test_data[0],names=['path','label','leisons'],sep='  ')

C:\Users\prais\AppData\Local\Temp\ipykernel_21464\722665227.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  test_df = pd.read_csv(test_data[0],names=['path','label','leisons'],sep='  ')


In [14]:
test_df

,path,label,leisons
0,levle0_451.jpg,0,2
1,levle0_498.jpg,0,1
2,levle0_485.jpg,0,3
3,levle0_218.jpg,0,2
4,levle1_344.jpg,0,4
...,...,...,...
287,levle3_88.jpg,3,64
288,levle3_1.jpg,3,60
289,levle3_68.jpg,3,56
290,levle3_16.jpg,3,65


In [15]:
import torchvision

mean = (0.5, 0.5, 0.5)
std = (0.5, 0.5, 0.5)

transform = torchvision.transforms.Compose([torchvision.transforms.Resize((224, 224)),
                                            torchvision.transforms.RandomHorizontalFlip(p=0.5),  # Randomly flip images left-right
                            torchvision.transforms.RandomVerticalFlip(p=0.5),
                            torchvision.transforms.RandomRotation(degrees=15),
                                            torchvision.transforms.ElasticTransform(),
                                               torchvision.transforms.ToTensor(),
                                               torchvision.transforms.Normalize(mean, std)])

test_transform = torchvision.transforms.Compose([torchvision.transforms.Resize((224, 224)),
                                               torchvision.transforms.ToTensor(),
                                               torchvision.transforms.Normalize(mean, std)])

In [16]:
train_dataset = Classification_Dataset(x_train,data_path = "../data-collection/dataset/acne-severity/JPEGImages",transform=transform,training=True)
val_dataset = Classification_Dataset(x_val,data_path = "../data-collection/dataset/acne-severity/JPEGImages",transform=test_transform,training=True)
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=bs, shuffle=True,
    )

# create test_loader
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False)

In [17]:
testset = Classification_Dataset(test_df,data_path = "../data-collection/dataset/acne-severity/JPEGImages",transform=test_transform,training=True)
test_loader = torch.utils.data.DataLoader(
        testset, batch_size=1, shuffle=False,
    )

In [18]:
train_metrics = Metrics(["accuracy_score","f1_score"])
val_metrics = Metrics(["accuracy_score","f1_score"])

In [19]:
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
     
        self.cnn = torchvision.models.efficientnet_v2_m(pretrained=True)
        for param in self.cnn.parameters():
            param.requires_grad = True
        self.cnn.classifier = nn.Sequential(
      
        nn.Linear(self.cnn.classifier[1].in_features, 512),
        nn.Dropout(p=0.2),
        nn.ReLU(),
        nn.Linear(512, 128),
        nn.Dropout(p=0.2),
        nn.Linear(128, 64),
            nn.Linear(64, 4),     
       )
        
    def forward(self, img):
        output = self.cnn(img)
        return output

In [20]:
model = MyNet()

c:\Users\prais\Workspace\AAIML\Sem1\MLFoundations\CSCN8010-class-notebooks\venv\tensorflow_cpu\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\prais\Workspace\AAIML\Sem1\MLFoundations\CSCN8010-class-notebooks\venv\tensorflow_cpu\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_M_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_M_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [21]:
a= torch.ones((16,3,224,224))

model(a)

tensor([[-0.0281,  0.0156,  0.0624, -0.0357],
        [-0.0294,  0.0376,  0.0323, -0.0361],
        [-0.0395,  0.0312,  0.0737, -0.0174],
        [-0.0129,  0.0362,  0.0530, -0.0089],
        [-0.0645,  0.0205,  0.0691, -0.0221],
        [-0.0346,  0.0478,  0.0862, -0.0091],
        [ 0.0002, -0.0145,  0.0297, -0.0296],
        [-0.0420,  0.0294,  0.0633, -0.0138],
        [-0.0422,  0.0243,  0.0829,  0.0174],
        [-0.0245,  0.0078,  0.0496, -0.0455],
        [-0.0442,  0.0306,  0.0611, -0.0483],
        [-0.0473,  0.0219,  0.0683, -0.0091],
        [-0.0275,  0.0421,  0.0619, -0.0155],
        [-0.0723,  0.0208,  0.0802, -0.0171],
        [-0.0230,  0.0296,  0.0608, -0.0578],
        [-0.0385,  0.0338,  0.0639, -0.0258]], grad_fn=<AddmmBackward0>)

In [22]:
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, "min", patience=4, factor=0.5
    )

In [23]:
from tqdm import tqdm
for param in model.parameters():
    param.requires_grad = True

num_epoch = 20
best_val_acc = 0.0
import logging
import numpy as np
print("begin training process")
for i in tqdm(range(0, num_epoch)):
    loss, val_loss, train_result, val_result = train_one_epoch(
        model,
        train_loader,
        val_loader,
        optimizer,
        criterion,
        train_metrics,
        val_metrics,
    )

    scheduler.step(val_loss)
#     scheduler.step()
    print(
        "Epoch {} / {} \n Training loss: {} - Other training metrics: ".format(
            i + 1, num_epoch, loss
        )
    )
    print(train_result)
    print(
        " \n Validation loss : {} - Other validation metrics:".format(val_loss)
    )
    print(val_result)
    print("\n")
    # saving epoch with best validation accuracy
    if (loss<0.04):
        # no saving
        continue
    if best_val_acc < float(val_result["accuracy_score"]):
        print(
            "Validation accuracy= "+
            str(val_result["accuracy_score"])+
            "===> Save best epoch"
        )
        best_val_acc = val_result["accuracy_score"]
        torch.save(
            model,
            "./" +  "best.pt"
        )
    else:
        print(
            "Validation accuracy= "+ str(val_result["accuracy_score"])+ "===> No saving"
        )
        continue

begin training process


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1 / 20 
 Training loss: 0.8047004469986125 - Other training metrics: 
{'accuracy_score': 0.4227467811158798, 'f1_score': 0.25096319572625064}
 
 Validation loss : 0.6689571408882673 - Other validation metrics:
{'accuracy_score': 0.5064377682403434, 'f1_score': 0.3217659533449007}


Validation accuracy= 0.5064377682403434===> Save best epoch


  5%|▌         | 1/20 [05:42<1:48:18, 342.02s/it]

Epoch 2 / 20 
 Training loss: 0.6273431765163405 - Other training metrics: 
{'accuracy_score': 0.6008583690987125, 'f1_score': 0.4454368803707521}
 
 Validation loss : 0.5125923054732479 - Other validation metrics:
{'accuracy_score': 0.6781115879828327, 'f1_score': 0.6321868692944117}


Validation accuracy= 0.6781115879828327===> Save best epoch


...

 90%|█████████ | 18/20 [1:40:08<10:58, 329.16s/it]

Epoch 18 / 20 
 Training loss: 0.08290231240265743 - Other training metrics: 
{'accuracy_score': 0.9624463519313304, 'f1_score': 0.9651206966170065}
 
 Validation loss : 0.47945385004189905 - Other validation metrics:
{'accuracy_score': 0.7682403433476395, 'f1_score': 0.6996395415899238}


Validation accuracy= 0.7682403433476395===> No saving


 95%|█████████▌| 19/20 [1:45:39<05:29, 329.72s/it]

Epoch 19 / 20 
 Training loss: 0.05064674750242239 - Other training metrics: 
{'accuracy_score': 0.9774678111587983, 'f1_score': 0.9805298821710972}
 
 Validation loss : 0.3978147612237112 - Other validation metrics:
{'accuracy_score': 0.7854077253218884, 'f1_score': 0.7572505195948658}


Validation accuracy= 0.7854077253218884===> No saving
Epoch 20 / 20 
 Training loss: 0.07171264170497144 - Other training metrics: 
{'accuracy_score': 0.9635193133047211, 'f1_score': 0.9692297927986743}
 
 Validation loss : 0.4140832553030084 - Other validation metrics:
{'accuracy_score': 0.7982832618025751, 'f1_score': 0.7571345237760115}


Validation accuracy= 0.7982832618025751===> Save best epoch


100%|██████████| 20/20 [1:51:11<00:00, 333.56s/it]


### FOLD 0

In [28]:
def test_result(model, test_loader,name='no_tta_prob.npy'):
    # testing the model by turning model "Eval" mode
    model.eval()
    preds = []
    aprobs = []
    labels = []
    with torch.no_grad():
        for data,target in test_loader:
            # forward-pass: compute-predicted-outputs-by-passing-inputs-to-the-model
            output = model(data)
            prob = nn.Softmax(dim=1)
            # applying Softmax to results
            probs = prob(output)
            aprobs.append(probs.cpu())
            labels.append(target.cpu().numpy())
            preds.extend(torch.argmax(probs, axis=1).tolist())
    aprobs = np.array(aprobs)
    np.save(name,aprobs)
    return preds, np.array(labels)

In [27]:
test_model = torch.load("best.pt")

In [29]:
preds,labels =test_result(test_model, test_loader)

C:\Users\prais\AppData\Local\Temp\ipykernel_21464\1345960528.py:17: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  aprobs = np.array(aprobs)
C:\Users\prais\AppData\Local\Temp\ipykernel_21464\1345960528.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  aprobs = np.array(aprobs)


In [30]:
from sklearn.metrics import classification_report as rp
print(rp(labels,preds))

              precision    recall  f1-score   support

           0       0.85      0.77      0.81       103
           1       0.72      0.82      0.76       127
           2       0.40      0.39      0.39        36
           3       0.74      0.54      0.62        26

    accuracy                           0.72       292
   macro avg       0.68      0.63      0.65       292
weighted avg       0.73      0.72      0.72       292



### FOLD 1

In [31]:
train_df = pd.read_csv(train_data[1],names=['path','label','leisons'],sep='  ')

C:\Users\prais\AppData\Local\Temp\ipykernel_21464\3807167902.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  train_df = pd.read_csv(train_data[1],names=['path','label','leisons'],sep='  ')


In [32]:
train_df

,path,label,leisons
0,levle0_0.jpg,0,3
1,levle0_1.jpg,0,3
2,levle0_100.jpg,0,2
3,levle0_101.jpg,0,1
4,levle0_102.jpg,0,3
...,...,...,...
1160,levle3_9.jpg,3,65
1161,levle3_91.jpg,3,58
1162,levle3_93.jpg,3,61
1163,levle3_95.jpg,3,59


In [33]:
x_train, x_val, y_train, y_val = data_split(train_df,0.2)

In [34]:
test_df = pd.read_csv(test_data[1],names=['path','label','leisons'],sep='  ')

C:\Users\prais\AppData\Local\Temp\ipykernel_21464\4252433160.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  test_df = pd.read_csv(test_data[1],names=['path','label','leisons'],sep='  ')


In [35]:
test_df

,path,label,leisons
0,levle0_60.jpg,0,1
1,levle0_483.jpg,0,2
2,levle0_110.jpg,0,2
3,levle0_404.jpg,0,1
4,levle0_155.jpg,0,1
...,...,...,...
287,levle3_37.jpg,3,55
288,levle3_109.jpg,3,52
289,levle3_76.jpg,3,56
290,levle3_113.jpg,3,64


In [36]:
mean = (0.5, 0.5, 0.5)
std = (0.5, 0.5, 0.5)
transform = torchvision.transforms.Compose([torchvision.transforms.Resize((224, 224)),
                                            torchvision.transforms.RandomHorizontalFlip(p=0.5),  # Randomly flip images left-right
                            torchvision.transforms.RandomVerticalFlip(p=0.5),
                            torchvision.transforms.RandomRotation(degrees=15),
                                            torchvision.transforms.ElasticTransform(),
                                               torchvision.transforms.ToTensor(),
                                               torchvision.transforms.Normalize(mean, std)])
test_transform = torchvision.transforms.Compose([torchvision.transforms.Resize((224, 224)),
                                               torchvision.transforms.ToTensor(),
                                               torchvision.transforms.Normalize(mean, std)])

In [37]:
train_dataset = Classification_Dataset(x_train,data_path = "../data-collection/dataset/acne-severity/JPEGImages",transform=transform,training=True)
val_dataset = Classification_Dataset(x_val,data_path = "../data-collection/dataset/acne-severity/JPEGImages",transform=test_transform,training=True)
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=bs, shuffle=True,
    )
    # create test_loader
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False)

In [38]:
testset = Classification_Dataset(test_df,data_path = "../data-collection/dataset/acne-severity/JPEGImages",transform=test_transform,training=True)
test_loader = torch.utils.data.DataLoader(
        testset, batch_size=1, shuffle=False,
    )

In [39]:
train_metrics = Metrics(["accuracy_score","f1_score"])
val_metrics = Metrics(["accuracy_score","f1_score"])

In [40]:
for param in model.parameters():
    param.requires_grad = True

num_epoch = 10
best_val_acc = 0.0
import logging
import numpy as np
print("begin training process")
for i in tqdm(range(0, num_epoch)):
    loss, val_loss, train_result, val_result = train_one_epoch(
        model,
        train_loader,
        val_loader,
        optimizer,
        criterion,
        train_metrics,
        val_metrics,
    )

    scheduler.step(val_loss)
#     scheduler.step()
    print(
        "Epoch {} / {} \n Training loss: {} - Other training metrics: ".format(
            i + 1, num_epoch, loss
        )
    )
    print(train_result)
    print(
        " \n Validation loss : {} - Other validation metrics:".format(val_loss)
    )
    print(val_result)
    print("\n")
    # saving epoch with best validation accuracy
    if (loss<0.04):
        # no saving
        continue
    if best_val_acc < float(val_result["accuracy_score"]):
        print(
            "Validation accuracy= "+
            str(val_result["accuracy_score"])+
            "===> Save best epoch"
        )
        best_val_acc = val_result["accuracy_score"]
        torch.save(
            model,
            "./" +  "best.pt"
        )
    else:
        print(
            "Validation accuracy= "+ str(val_result["accuracy_score"])+ "===> No saving"
        )
        continue

begin training process


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1 / 10 
 Training loss: 0.23553761654401542 - Other training metrics: 
{'accuracy_score': 0.8798283261802575, 'f1_score': 0.8639048489625796}
 
 Validation loss : 0.17254476741659794 - Other validation metrics:
{'accuracy_score': 0.9012875536480687, 'f1_score': 0.8963823655264417}


Validation accuracy= 0.9012875536480687===> Save best epoch


 20%|██        | 2/10 [14:39<57:25, 430.66s/it]  

Epoch 2 / 10 
 Training loss: 0.17336342029689208 - Other training metrics: 
{'accuracy_score': 0.9066523605150214, 'f1_score': 0.8988772469519628}
 
 Validation loss : 0.19005355597053986 - Other validation metrics:
{'accuracy_score': 0.8927038626609443, 'f1_score': 0.8811063709128073}


Validation accuracy= 0.8927038626609443===> No saving


...

 80%|████████  | 8/10 [47:07<11:05, 332.65s/it]

Epoch 8 / 10 
 Training loss: 0.0702551011587673 - Other training metrics: 
{'accuracy_score': 0.973175965665236, 'f1_score': 0.9785981150445221}
 
 Validation loss : 0.14229219234168786 - Other validation metrics:
{'accuracy_score': 0.9184549356223176, 'f1_score': 0.9144462818104977}


Validation accuracy= 0.9184549356223176===> Save best epoch


 90%|█████████ | 9/10 [52:39<05:32, 332.44s/it]

Epoch 9 / 10 
 Training loss: 0.07076271866396261 - Other training metrics: 
{'accuracy_score': 0.9656652360515021, 'f1_score': 0.9679728224178784}
 
 Validation loss : 0.1498966107840446 - Other validation metrics:
{'accuracy_score': 0.9184549356223176, 'f1_score': 0.9189975334742776}


Validation accuracy= 0.9184549356223176===> No saving
Epoch 10 / 10 
 Training loss: 0.05902129135897999 - Other training metrics: 
{'accuracy_score': 0.9753218884120172, 'f1_score': 0.9735185727335947}
 
 Validation loss : 0.1511761919623281 - Other validation metrics:
{'accuracy_score': 0.9227467811158798, 'f1_score': 0.9223620127477872}


Validation accuracy= 0.9227467811158798===> Save best epoch


100%|██████████| 10/10 [57:59<00:00, 347.96s/it]


In [41]:
test_model = torch.load("best.pt")

In [42]:
preds_1,labels_1 = test_result(test_model, test_loader)

C:\Users\prais\AppData\Local\Temp\ipykernel_21464\1345960528.py:17: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  aprobs = np.array(aprobs)
C:\Users\prais\AppData\Local\Temp\ipykernel_21464\1345960528.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  aprobs = np.array(aprobs)


In [43]:
print(rp(preds_1,labels_1))

              precision    recall  f1-score   support

           0       0.98      0.94      0.96       107
           1       0.94      0.98      0.96       122
           2       0.92      0.89      0.90        37
           3       0.92      0.92      0.92        26

    accuracy                           0.95       292
   macro avg       0.94      0.93      0.94       292
weighted avg       0.95      0.95      0.95       292



### FOLD 2

In [44]:
train_df = pd.read_csv(train_data[2],names=['path','label','leisons'],sep='  ')

C:\Users\prais\AppData\Local\Temp\ipykernel_21464\2371486086.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  train_df = pd.read_csv(train_data[2],names=['path','label','leisons'],sep='  ')


In [45]:
train_df

,path,label,leisons
0,levle0_0.jpg,0,3
1,levle0_100.jpg,0,2
2,levle0_101.jpg,0,1
3,levle0_102.jpg,0,3
4,levle0_103.jpg,0,2
...,...,...,...
1160,levle3_93.jpg,3,61
1161,levle3_94.jpg,3,57
1162,levle3_95.jpg,3,51
1163,levle3_96.jpg,3,55


In [46]:
x_train, x_val, y_train, y_val = data_split(train_df,0.2)

In [47]:
test_df = pd.read_csv(test_data[2],names=['path','label','leisons'],sep='  ')

C:\Users\prais\AppData\Local\Temp\ipykernel_21464\1489905983.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  test_df = pd.read_csv(test_data[2],names=['path','label','leisons'],sep='  ')


In [48]:
test_df

,path,label,leisons
0,levle0_426.jpg,0,2
1,levle0_382.jpg,0,4
2,levle0_517.jpg,0,3
3,levle0_115.jpg,0,1
4,levle0_338.jpg,0,1
...,...,...,...
287,levle3_26.jpg,3,55
288,levle3_87.jpg,3,56
289,levle3_19.jpg,3,57
290,levle3_9.jpg,3,65


In [49]:
mean = (0.5, 0.5, 0.5)
std = (0.5, 0.5, 0.5)
transform = torchvision.transforms.Compose([torchvision.transforms.Resize((224, 224)),
                                            torchvision.transforms.RandomHorizontalFlip(p=0.5),  # Randomly flip images left-right
                            torchvision.transforms.RandomVerticalFlip(p=0.5),
                            torchvision.transforms.RandomRotation(degrees=15),
                                            torchvision.transforms.ElasticTransform(),
                                               torchvision.transforms.ToTensor(),
                                               torchvision.transforms.Normalize(mean, std)])
test_transform = torchvision.transforms.Compose([torchvision.transforms.Resize((224, 224)),
                                               torchvision.transforms.ToTensor(),
                                               torchvision.transforms.Normalize(mean, std)])

In [50]:
train_dataset = Classification_Dataset(x_train,data_path = "../data-collection/dataset/acne-severity/JPEGImages",transform=transform,training=True)
val_dataset = Classification_Dataset(x_val,data_path = "../data-collection/dataset/acne-severity/JPEGImages",transform=test_transform,training=True)
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=bs, shuffle=True,
    )
    # create test_loader
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False)

In [51]:
testset = Classification_Dataset(test_df,data_path = "../data-collection/dataset/acne-severity/JPEGImages",transform=test_transform,training=True)
test_loader = torch.utils.data.DataLoader(
        testset, batch_size=1, shuffle=False,
    )

In [52]:
for param in model.parameters():
    param.requires_grad = True

num_epoch = 8
best_val_acc = 0.0
import logging
import numpy as np
print("begin training process")
for i in tqdm(range(0, num_epoch)):
    loss, val_loss, train_result, val_result = train_one_epoch(
        model,
        train_loader,
        val_loader,
        optimizer,
        criterion,
        train_metrics,
        val_metrics,
    )

    scheduler.step(val_loss)
#     scheduler.step()
    print(
        "Epoch {} / {} \n Training loss: {} - Other training metrics: ".format(
            i + 1, num_epoch, loss
        )
    )
    print(train_result)
    print(
        " \n Validation loss : {} - Other validation metrics:".format(val_loss)
    )
    print(val_result)
    print("\n")
    # saving epoch with best validation accuracy
    if (loss<0.04):
        # no saving
        continue
    if best_val_acc < float(val_result["accuracy_score"]):
        print(
            "Validation accuracy= "+
            str(val_result["accuracy_score"])+
            "===> Save best epoch"
        )
        best_val_acc = val_result["accuracy_score"]
        torch.save(
            model,
            "./" +  "best.pt"
        )
    else:
        print(
            "Validation accuracy= "+ str(val_result["accuracy_score"])+ "===> No saving"
        )
        continue

begin training process


  0%|          | 0/8 [00:00<?, ?it/s]

Epoch 1 / 8 
 Training loss: 0.11493869910872034 - Other training metrics: 
{'accuracy_score': 0.9399141630901288, 'f1_score': 0.933736726721598}
 
 Validation loss : 0.07604187256916398 - Other validation metrics:
{'accuracy_score': 0.9613733905579399, 'f1_score': 0.9593689096255255}


Validation accuracy= 0.9613733905579399===> Save best epoch


 25%|██▌       | 2/8 [10:40<31:59, 319.97s/it]

Epoch 2 / 8 
 Training loss: 0.08470647704943439 - Other training metrics: 
{'accuracy_score': 0.9603004291845494, 'f1_score': 0.9532924973351192}
 
 Validation loss : 0.0865488896074469 - Other validation metrics:
{'accuracy_score': 0.9570815450643777, 'f1_score': 0.952295918367347}


Validation accuracy= 0.9570815450643777===> No saving


...

Epoch 6 / 8 
 Training loss: 0.06763697720568451 - Other training metrics: 
{'accuracy_score': 0.9688841201716738, 'f1_score': 0.9736606610530041}
 
 Validation loss : 0.08622069618476819 - Other validation metrics:
{'accuracy_score': 0.9570815450643777, 'f1_score': 0.9565940574451854}


Validation accuracy= 0.9570815450643777===> No saving


 88%|████████▊ | 7/8 [2:20:21<28:55, 1735.24s/it]

Epoch 7 / 8 
 Training loss: 0.0609141271121704 - Other training metrics: 
{'accuracy_score': 0.9656652360515021, 'f1_score': 0.9678269098030382}
 
 Validation loss : 0.07849532688764031 - Other validation metrics:
{'accuracy_score': 0.9570815450643777, 'f1_score': 0.9522822299651568}


Validation accuracy= 0.9570815450643777===> No saving


100%|██████████| 8/8 [2:57:52<00:00, 1334.01s/it]

Epoch 8 / 8 
 Training loss: 0.0448160262762413 - Other training metrics: 
{'accuracy_score': 0.9785407725321889, 'f1_score': 0.9793734671536439}
 
 Validation loss : 0.0864113770872994 - Other validation metrics:
{'accuracy_score': 0.9613733905579399, 'f1_score': 0.9623278406414336}


Validation accuracy= 0.9613733905579399===> No saving


In [53]:
test_model = torch.load("best.pt")

In [54]:
preds_2,labels_2 =test_result(test_model, test_loader)

C:\Users\prais\AppData\Local\Temp\ipykernel_21464\1345960528.py:17: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  aprobs = np.array(aprobs)
C:\Users\prais\AppData\Local\Temp\ipykernel_21464\1345960528.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  aprobs = np.array(aprobs)


In [ ]:
print(rp(preds_2,labels_2))

### FOLD 3

In [55]:
train_df = pd.read_csv(train_data[3],names=['path','label','leisons'],sep='  ')

C:\Users\prais\AppData\Local\Temp\ipykernel_21464\975574921.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  train_df = pd.read_csv(train_data[3],names=['path','label','leisons'],sep='  ')


In [56]:
train_df

,path,label,leisons
0,levle0_1.jpg,0,3
1,levle0_100.jpg,0,2
2,levle0_102.jpg,0,3
3,levle0_103.jpg,0,2
4,levle0_104.jpg,0,1
...,...,...,...
1160,levle3_91.jpg,3,52
1161,levle3_92.jpg,3,57
1162,levle3_94.jpg,3,65
1163,levle3_96.jpg,3,57


In [57]:
x_train, x_val, y_train, y_val = data_split(train_df,0.2)

In [58]:
test_df

,path,label,leisons
0,levle0_426.jpg,0,2
1,levle0_382.jpg,0,4
2,levle0_517.jpg,0,3
3,levle0_115.jpg,0,1
4,levle0_338.jpg,0,1
...,...,...,...
287,levle3_26.jpg,3,55
288,levle3_87.jpg,3,56
289,levle3_19.jpg,3,57
290,levle3_9.jpg,3,65


In [59]:
train_dataset = Classification_Dataset(x_train,data_path = "../data-collection/dataset/acne-severity/JPEGImages",transform=transform,training=True)
val_dataset = Classification_Dataset(x_val,data_path = "../data-collection/dataset/acne-severity/JPEGImages",transform=test_transform,training=True)
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=bs, shuffle=True,
    )
    # create test_loader
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False)

In [60]:
testset = Classification_Dataset(test_df,data_path = "../data-collection/dataset/acne-severity/JPEGImages",transform=test_transform,training=True)
test_loader = torch.utils.data.DataLoader(
        testset, batch_size=1, shuffle=False,
    )

In [61]:
train_metrics = Metrics(["accuracy_score","f1_score"])
val_metrics = Metrics(["accuracy_score","f1_score"])

In [64]:
for param in model.parameters():
    param.requires_grad = True

num_epoch = 8
best_val_acc = 0.0
import logging
import numpy as np
print("begin training process")
for i in tqdm(range(0, num_epoch)):
    loss, val_loss, train_result, val_result = train_one_epoch(
        model,
        train_loader,
        val_loader,
        optimizer,
        criterion,
        train_metrics,
        val_metrics,
    )

    scheduler.step(val_loss)
#     scheduler.step()
    print(
        "Epoch {} / {} \n Training loss: {} - Other training metrics: ".format(
            i + 1, num_epoch, loss
        )
    )
    print(train_result)
    print(
        " \n Validation loss : {} - Other validation metrics:".format(val_loss)
    )
    print(val_result)
    print("\n")
    # saving epoch with best validation accuracy
    if (loss<0.04):
        # no saving
        continue
    if best_val_acc < float(val_result["accuracy_score"]):
        print(
            "Validation accuracy= "+
            str(val_result["accuracy_score"])+
            "===> Save best epoch"
        )
        best_val_acc = val_result["accuracy_score"]
        torch.save(
            model,
            "./" +  "best.pt"
        )
    else:
        print(
            "Validation accuracy= "+ str(val_result["accuracy_score"])+ "===> No saving"
        )
        continue

begin training process


  0%|          | 0/8 [00:00<?, ?it/s]

Epoch 1 / 8 
 Training loss: 0.0501968581948925 - Other training metrics: 
{'accuracy_score': 0.9774678111587983, 'f1_score': 0.9769535920873927}
 
 Validation loss : 0.0316298113168565 - Other validation metrics:
{'accuracy_score': 0.9914163090128756, 'f1_score': 0.9870348121744628}


Validation accuracy= 0.9914163090128756===> Save best epoch


 25%|██▌       | 2/8 [10:34<31:45, 317.63s/it]

Epoch 2 / 8 
 Training loss: 0.05738462724567035 - Other training metrics: 
{'accuracy_score': 0.9688841201716738, 'f1_score': 0.9772371368127961}
 
 Validation loss : 0.033171262160391256 - Other validation metrics:
{'accuracy_score': 0.9914163090128756, 'f1_score': 0.9870348121744628}


Validation accuracy= 0.9914163090128756===> No saving


...

Epoch 6 / 8 
 Training loss: 0.0448490123328445 - Other training metrics: 
{'accuracy_score': 0.9796137339055794, 'f1_score': 0.9808956429066245}
 
 Validation loss : 0.03441809001078933 - Other validation metrics:
{'accuracy_score': 0.9914163090128756, 'f1_score': 0.9870348121744628}


Validation accuracy= 0.9914163090128756===> No saving


 88%|████████▊ | 7/8 [37:21<05:20, 320.90s/it]

Epoch 7 / 8 
 Training loss: 0.04145666221836222 - Other training metrics: 
{'accuracy_score': 0.9817596566523605, 'f1_score': 0.9809516049731535}
 
 Validation loss : 0.03362672676855914 - Other validation metrics:
{'accuracy_score': 0.9914163090128756, 'f1_score': 0.9870348121744628}


Validation accuracy= 0.9914163090128756===> No saving


100%|██████████| 8/8 [42:42<00:00, 320.26s/it]

Epoch 8 / 8 
 Training loss: 0.043006054335573585 - Other training metrics: 
{'accuracy_score': 0.9839055793991416, 'f1_score': 0.9876023999728817}
 
 Validation loss : 0.03670686150825075 - Other validation metrics:
{'accuracy_score': 0.9914163090128756, 'f1_score': 0.9870348121744628}


Validation accuracy= 0.9914163090128756===> No saving


In [65]:
test_model = torch.load("best.pt")
preds_3,labels_3 =test_result(test_model, test_loader)

C:\Users\prais\AppData\Local\Temp\ipykernel_21464\1345960528.py:17: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  aprobs = np.array(aprobs)
C:\Users\prais\AppData\Local\Temp\ipykernel_21464\1345960528.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  aprobs = np.array(aprobs)


In [66]:
print(rp(preds_3,labels_3))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       101
           1       1.00      0.98      0.99       130
           2       0.94      1.00      0.97        34
           3       1.00      0.96      0.98        27

    accuracy                           0.99       292
   macro avg       0.98      0.98      0.98       292
weighted avg       0.99      0.99      0.99       292



### FOLD 4

In [67]:
train_df = pd.read_csv(train_data[4],names=['path','label','leisons'],sep='  ')

C:\Users\prais\AppData\Local\Temp\ipykernel_21464\1650634807.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  train_df = pd.read_csv(train_data[4],names=['path','label','leisons'],sep='  ')


In [68]:
x_train, x_val, y_train, y_val = data_split(train_df,0.2)

In [69]:
test_df = pd.read_csv(test_data[4],names=['path','label','leisons'],sep='  ')

C:\Users\prais\AppData\Local\Temp\ipykernel_21464\4220881812.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  test_df = pd.read_csv(test_data[4],names=['path','label','leisons'],sep='  ')


In [70]:
train_dataset = Classification_Dataset(x_train,data_path = "../data-collection/dataset/acne-severity/JPEGImages",transform=transform,training=True)
val_dataset = Classification_Dataset(x_val,data_path = "../data-collection/dataset/acne-severity/JPEGImages",transform=test_transform,training=True)
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=bs, shuffle=True,
    )
    # create test_loader
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False)

In [71]:
testset = Classification_Dataset(test_df,data_path = "../data-collection/dataset/acne-severity/JPEGImages",transform=test_transform,training=True)
test_loader = torch.utils.data.DataLoader(
        testset, batch_size=1, shuffle=False,
    )

In [72]:
train_metrics = Metrics(["accuracy_score","f1_score"])
val_metrics = Metrics(["accuracy_score","f1_score"])

In [73]:
for param in model.parameters():
    param.requires_grad = True

num_epoch = 8
best_val_acc = 0.0
import logging
import numpy as np
print("begin training process")
for i in tqdm(range(0, num_epoch)):
    loss, val_loss, train_result, val_result = train_one_epoch(
        model,
        train_loader,
        val_loader,
        optimizer,
        criterion,
        train_metrics,
        val_metrics,
    )

    scheduler.step(val_loss)
#     scheduler.step()
    print(
        "Epoch {} / {} \n Training loss: {} - Other training metrics: ".format(
            i + 1, num_epoch, loss
        )
    )
    print(train_result)
    print(
        " \n Validation loss : {} - Other validation metrics:".format(val_loss)
    )
    print(val_result)
    print("\n")
    # saving epoch with best validation accuracy
    if (loss<0.04):
        # no saving
        continue
    if best_val_acc < float(val_result["accuracy_score"]):
        print(
            "Validation accuracy= "+
            str(val_result["accuracy_score"])+
            "===> Save best epoch"
        )
        best_val_acc = val_result["accuracy_score"]
        torch.save(
            model,
            "./" +  "best.pt"
        )
    else:
        print(
            "Validation accuracy= "+ str(val_result["accuracy_score"])+ "===> No saving"
        )
        continue

begin training process


  0%|          | 0/8 [00:00<?, ?it/s]

Epoch 1 / 8 
 Training loss: 0.049775041815676914 - Other training metrics: 
{'accuracy_score': 0.9763948497854077, 'f1_score': 0.9787720324161091}
 
 Validation loss : 0.0235369814631765 - Other validation metrics:
{'accuracy_score': 0.9914163090128756, 'f1_score': 0.9889040628200751}


Validation accuracy= 0.9914163090128756===> Save best epoch


 25%|██▌       | 2/8 [10:48<32:26, 324.44s/it]

Epoch 2 / 8 
 Training loss: 0.04119120356766707 - Other training metrics: 
{'accuracy_score': 0.9828326180257511, 'f1_score': 0.9843343250776654}
 
 Validation loss : 0.024493591995837862 - Other validation metrics:
{'accuracy_score': 0.9871244635193133, 'f1_score': 0.986150986732751}


Validation accuracy= 0.9871244635193133===> No saving


...

Epoch 6 / 8 
 Training loss: 0.0383300655201685 - Other training metrics: 
{'accuracy_score': 0.9871244635193133, 'f1_score': 0.9896696924307774}
 
 Validation loss : 0.02354261386228222 - Other validation metrics:
{'accuracy_score': 0.9914163090128756, 'f1_score': 0.9916293123642741}




 88%|████████▊ | 7/8 [37:42<05:22, 322.70s/it]

Epoch 7 / 8 
 Training loss: 0.03245136943993522 - Other training metrics: 
{'accuracy_score': 0.990343347639485, 'f1_score': 0.9911927348697956}
 
 Validation loss : 0.021016685428025896 - Other validation metrics:
{'accuracy_score': 0.9914163090128756, 'f1_score': 0.9889040628200751}




100%|██████████| 8/8 [43:04<00:00, 323.00s/it]

Epoch 8 / 8 
 Training loss: 0.03661893569972254 - Other training metrics: 
{'accuracy_score': 0.9860515021459227, 'f1_score': 0.9889802297133379}
 
 Validation loss : 0.0202250503035574 - Other validation metrics:
{'accuracy_score': 0.9957081545064378, 'f1_score': 0.9943825079941232}




In [74]:
test_model = torch.load("best.pt")

In [75]:
preds_4,labels_4 =test_result(test_model, test_loader)

C:\Users\prais\AppData\Local\Temp\ipykernel_21464\1345960528.py:17: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  aprobs = np.array(aprobs)
C:\Users\prais\AppData\Local\Temp\ipykernel_21464\1345960528.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  aprobs = np.array(aprobs)


In [76]:
print(rp(labels_4,preds_4))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00       103
           1       0.98      1.00      0.99       127
           2       1.00      0.94      0.97        36
           3       1.00      1.00      1.00        26

    accuracy                           0.99       292
   macro avg       0.99      0.98      0.99       292
weighted avg       0.99      0.99      0.99       292

